In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import h5py
import sys

sys.path.append('../')

from cnn.finetune_commons import show_images, ft_train_model, visualize_model
from cnn.commons import save_dict_to_hdf5

plt.ion()   # interactive mode

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
torch.manual_seed(78)
np.random.seed(78)

data_dir = '../data/chest_xray'
# data_dir = '../data/rsna-bone-age'

n_labels = 2

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        #bone
        #transforms.Normalize([0.18805075, 0.18805075, 0.18805075], [0.1892409, 0.1892409, 0.1892409])
        
        #chest
        transforms.Normalize([0.5709945, 0.5709945, 0.5709945], [0.1767184, 0.1767184, 0.1767184])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        #bone
        #transforms.Normalize([0.18805075, 0.18805075, 0.18805075], [0.1892409, 0.1892409, 0.1892409])
        
        #chest
        transforms.Normalize([0.5709945, 0.5709945, 0.5709945], [0.1767184, 0.1767184, 0.1767184])        
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x]) for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                        shuffle=True, num_workers=4) for x in ['train', 'test']}

In [4]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### VGG16

In [5]:
model_ft = models.vgg16(pretrained=True)
model_ft = model_ft.eval()
for param in model_ft.features.parameters():
    param.requires_grad = False

num_ftrs = model_ft.classifier[-1].in_features
temp = list(model_ft.classifier.children())[:-1]
temp.append(nn.Linear(num_ftrs, n_labels))
model_ft.classifier = nn.Sequential(*temp)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(model_ft.classifier.parameters(), lr=1e-4, weight_decay=1e-2)

In [ ]:
model_ft = ft_train_model(model_ft, criterion, optimizer_ft, dataloaders, device,
                          dataset_sizes, class_names, num_epochs=50)

Epoch 0/49
----------
train Loss: 0.1315 Acc: 0.9471
test Loss: 0.6676 Acc: 0.8141
()
Epoch 1/49
----------
train Loss: 0.0763 Acc: 0.9731
test Loss: 1.0710 Acc: 0.7644
()
Epoch 2/49
----------
train Loss: 0.0670 Acc: 0.9763
test Loss: 0.7573 Acc: 0.7933
()
Epoch 3/49
----------
train Loss: 0.0426 Acc: 0.9841
test Loss: 0.7486 Acc: 0.8285
()
Epoch 4/49
----------
train Loss: 0.0373 Acc: 0.9870
test Loss: 0.9034 Acc: 0.8141
()
Epoch 5/49
----------
train Loss: 0.0447 Acc: 0.9830
test Loss: 0.4940 Acc: 0.8814
()
Epoch 6/49
----------
train Loss: 0.0300 Acc: 0.9878
test Loss: 1.0477 Acc: 0.7772
()
Epoch 7/49
----------
train Loss: 0.0344 Acc: 0.9870
test Loss: 1.3089 Acc: 0.7660
()
Epoch 8/49
----------
train Loss: 0.0293 Acc: 0.9895
test Loss: 0.9027 Acc: 0.8093
()
Epoch 9/49
----------
